In [99]:
"#!/usr/bin/env python"
#-*- coding: utf-8- -*-
import warnings 
warnings.filterwarnings("ignore")
%load_ext pycodestyle_magic
%pycodestyle_on
#load prerequisite librarys  for running the code 
from osgeo import gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapely

#%matplotlib

In [262]:
class TPI_Land_Form_Classification:
    """
    Class prefroming automated landform recognition and transposition from raster to vector layers
    """
    def __init__(self , elevation_model = "",output_model = "lf_output.tif" , TPI_Radius_m_lst = []):
        """
        Base init Function
        - TPI_Radius_m_lst: list of tpi values to comput for land feature recognition
        - elevation_model: str the directory for the input elevation_modela
        """
        """
        TODO : add all of these natively t
        #!/usr/bin/env python
        #-*- coding: utf-8- -*-
        import warnings 
        warnings.filterwarnings("ignore")

        #load prerequisite librarys  for running the code 
        from osgeo import gdal
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        import shapely"""
        
        self.elevation_model = elevation_model
        self.output_model = output_model
        self.TPI_Radius_m_lst = TPI_Radius_m_lst
        
        assert type(TPI_Radius_m_lst) == list , "Please enter a list of integer values to TPI_Radius_m_lst"
        assert len(elevation_model) > 0 , "please enter a valid directory string leading to a GeoTiff file"
        
    def get_raster_data(self, elevation_model):
        """
        Function returning numpy array from elevation_model along with resolution of said array
         - elevation_model: str the directory for the input elevation_model
        """
        #get raster base data
        dem_data = gdal.Open(elevation_model)
        #set raster data as array
        #dem_array = dem_data.GetRasterBand(1).ReadAsArray()
        #get the geotransform data from the raster
        transform = dem_data.GetGeoTransform()
        #get projection data
        projection = dem_data.GetProjection()
        #get the rasters resolution
        resolution = transform[1]
        
        return projection ,transform, resolution, dem_array
    
    
    def write_output(self, output_model, output_data, original_transform, original_projection):
        """
        Function writing array to GeoTiff file in user specified directory
        - output_model: str the directory for the output model
        - output_data: numpy array that the user defines
        - original_transform: tupel original raster geotransform
        - original_projection: tupel original raster projection
        """
        # set driver
        driver = gdal.GetDriverByName('GTiff')
        #set output driver and instance new raster
        ds = driver.Create(output_model, output_data.shape[1], output_data.shape[0], 1, gdal.GDT_Float32) 
        #set new raster metadata by old raster metadata
        ds.SetProjection(dem.GetProjection())
        ds.SetGeoTransform(dem.GetGeoTransform())
        ds.GetRasterBand(1).WriteArray(out)
        #remove writen raster from memory
        ds.FlushCache()
        ds = None
        
    #with numpy.mean()
    def downsample_res_avg(self, array, scale):
        """
        Function downsampling elevation model by input scale value, aggrigating values by average
        - array: elevation model as ndarray
        - scale: the value by which the user would like to simplify the input array
        """
        original_width = array.shape[1]
        original_hight = array.shape[0]

        width = int( original_width / scale)
        hight = int(original_hight / scale)

        resized_array = np.zeros(shape = (hight, width, 3) ,dtype = np.uint8)

        for i in range(0, original_hight , scale):
            for j in range(0, original_width , scale):
                resized_array[int(i/scale)-1 , int(j/scale)-1 ] = np.mean(array[i:(i + scale), j:(j + scale)], axis = (0,1))

        return resized_array

    
    def get_hillshade(self, slope_array, aspect_array, azimuth = "90", angle_altitude = 100, ):
        """
        Function hillshade for each cell in input elevation model
        - slope_array: elevation model slope values as ndarray
        - aspect_array:  elevation model aspect values as ndarray
        - azimuth: azimuth direction for hillshade algorithm calculation
        - angle_altitude: altitude for angle calculation in hillshade algorithm
        """
        azimuth = 360 - azimuth
        azimuth_rad = azimuth *np.pi/180
        altitude_rad = angle_altitude*np.pi/180
        shade = np.sin(altitude_rad)*np.sin(slope) + np.cos(altitude_rad)*np.cos(slope)*np.cos((azimuth - np.pi/2)  - aspect)
        hillshade = 255 * (shaded + 1)/2
        return hillshade
        

    
    def get_slope_and_aspect(self,array, in_degrees = True ):
        """
        Function calculating slope and aspect for each cell in input elevation model
        - array: elevation model as ndarray
        - in_degrees: boolean switch for returning slope values ini degrees or decimals
        """
        x,y = np.gradient(array)
        slope = np.pi/2 - np.arctan(np.sqrt(x*x + y*y))
        aspect = np.arctan2(-x,y)
        if in_degrees is True:
            slope = np.degrees(np.arctan(slope))
        return slope, aspect
    
    def view (self, offset_y, offset_x, shape, step = 1):
        """
        Function returning two matching numpy views for moving window routines.
        - 'offset_y' and 'offset_x' refer to the shift in relation to the analysed (central) cell 
        - 'shape' are 2 dimensions of the data matrix (not of the window!)
        - 'view_in' is the shifted view and 'view_out' is the position of central cells
        """
        size_y, size_x = shape
        x, y = abs(offset_x), abs(offset_y)

        x_in = slice(x , size_x, step) 
        x_out = slice(0, size_x - x, step)

        y_in = slice(y, size_y, step)
        y_out = slice(0, size_y - y, step)

        # the swapping trick    
        if offset_x < 0: x_in, x_out = x_out, x_in                                 
        if offset_y < 0: y_in, y_out = y_out, y_in

        # return window view (in) and main view (out)
        return np.s_[y_in, x_in], np.s_[y_out, x_out]

        
    def calculate_TPI(self, dem_array, radius_m, resolution):
        """
        Function calculating TPI value of dem array at input radius 
        - dem_array: elevation model as numpy array
        - radius_m: radius in meters for TPI calculation
        - resolution: Resolution of original elevation model
        """
        #radius in pixels
        r = int(np.floor(radius_m/resolution))
        #instance window
        win = np.ones((2* r +1, 2* r +1))
        # win = np.array( [    [0, 1, 1, 1, 0]
        #                      [1, 1, 1, 1, 1],
        #                      [1, 1, 0, 1, 1],
        #                      [1, 1, 1, 1, 1],
        #                      [0, 1, 1, 1, 0]  ])

        # window radius is needed for the function,
        # deduce from window size (can be different for height and width…)
        r_y, r_x  = win.shape[0]//2, win.shape[1]//2
        win[r_y, r_x  ] = 0  # let's remove the central cell 

        #matrices for temporary data
        mx_temp = np.zeros(dem_array.shape)
        mx_count = np.zeros(dem_array.shape)

        # loop through window and accumulate values
        for (y,x), weight in np.ndenumerate(win):

            if weight == 0 : continue  #skip zero values !
            # determine views to extract data 
            view_in, view_out = self.view(y - r_y, x - r_x, dem_array.shape)
            # using window weights (eg. for a Gaussian function)
            mx_temp[view_out] += dem_array[view_in]  * weight

           # track the number of neighbours 
           # (this is used for weighted mean : Σ weights*val / Σ weights)
            mx_count[view_out] += weight

        # this is TPI (spot height – average neighbourhood height)
        out = dem_array - mx_temp / mx_count
        #normalize output
        out = ((((out - out.mean()) / out.std()) * 100) + 0.5)
        return out
    
    class Primes():
        """
        class
        """
        math = __import__('math')
        def __init__(self,listLength = int):
            """
            func
            """
            assert type(listLength) is int , "please enter an integer to listLength"
            self.listLength = listLength

        def isPrime (self, primeList, candidate):
            """
            func
            """
            upperLimit = self.math.sqrt(candidate)
            for p in primeList:
                if candidate % p == 0:
                    return False
                if p >= upperLimit:
                    break

            return True

        def primeList(self):
            """
            func
            """
            if self.listLength < 1 :
                return []
            primes = [2]

            candidate = 3
            while self.listLength > len(primes):
                if self.isPrime(primes, candidate):
                    primes.append(candidate)
                candidate +=2

            return primes
    
    def add_prime_labels(self, tpi_array_dict, slope): 
        tpi_array_lst = list(tpi_array_dict.values()) 
        primes = self.Primes(listLength= int(4*(len(tpi_array_lst) +1)))
        primeList = primes.primeList()

        legend_dict = {"peak":{"heirarchy" : []}, "valley" : {"heirarchy" : []}, "slope" : {"heirarchy" : []} , "plane": {"heirarchy" : []}}

        std = tpi_array_lst[-1].std()
        lf_array = np.full(tpi_array_lst[-1].shape, 1)

        for index, tpi in enumerate(tpi_array_lst):
            print(f"index number {index} started")
            prime1, prime2, prime3, prime4 = primeList[index * 4:(index + 1)* 4]
            # add to legend dictionary 

            lf_array[tpi >= std] = lf_array[tpi > std] * prime4
            legend_dict["peak"]["heirarchy"].append(prime4)

            lf_array[tpi <= -std] = lf_array[tpi < -std] * prime1 
            legend_dict["valley"]["heirarchy"].append(prime1)

            lf_array[(tpi > -std) & (tpi < std ) & (slope >= 6)] = lf_array[(tpi > -std) & (tpi < std ) & (slope >= 6)] * prime2 
            legend_dict["slope"]["heirarchy"].append(prime2)

            lf_array[(tpi > -std) & (tpi < std ) & (slope <= 5)]= lf_array[(tpi > -std) & (tpi < std ) & (slope <= 5)] * prime3
            legend_dict["plane"]["heirarchy"].append(prime3)

            print(f"index number {index} complete")
        return lf_array ,legend_dict
    
   
    
    """def visualize(self, array,hillshade):
        
        Function visualizing the output from the TPI_Land_Form_Classification algorithm
        - array: landfrom class labeled array
        - hillshade: hillshade claculation output ndarray
        
        %matplotlib
        
        # blend hillshade and landform class arrays
        blend = array*0.5 + hillshade*0.5
        #print array as 
        plt.imshow(array)
        plt.show()
        
        return"""
    
    def test_fit(self):
        """
        Funcution running a mokup of the TPI_Land_Form_Classification algorithm
        """
        #get dem data
        #projection ,transform, resolution, dem_array = self.get_raster_data(elevation_model = self.elevation_model)
        
        #initailize thread pool
        """from multiprocessing.pool import ThreadPool
        from functools import partial
        threads = len(self.TPI_Radius_m_lst)
        t = ThreadPool(threads)"""
        
        # get each array for block by block calculation by the tpi to be run?
        # make sure to inlarge it only in the direction where there are values.
        # export only from the given window and copy nan values faithfully so as to remove 
        # unusable data.
        # Function to read the raster as arrays for the chosen block size.
        
        
        #get raster base data
        ds = gdal.Open(elevation_model)
        #get raster and data
        band = dem_data.GetRasterBand(1)
        # Get "natural" block size, and total raster XY size. 
        block_sizes = band.GetBlockSize()
        x_block_size = block_sizes[0]
        y_block_size = block_sizes[1]
        """replace with gcd?"""
        
        #add band and array size
        xsize = band.XSize
        ysize = band.YSize

        #get the geotransform data from the raster
        geoT = dem_data.GetGeoTransform()
        #get projection data
        projection = dem_data.GetProjection()
        #get the rasters resolution
        resolution = geoT[1]

        #get projection reference
        raster_srs = osr.SpatialReference()
        raster_srs.ImportFromWkt(ds.GetProjectionRef())
        #----------------------------------------------------------------
        #set export data
        # set driver
        driver = gdal.GetDriverByName('GTiff')
        #set output driver and instance new raster
        dst_ds = driver.Create(output_model, xsize, ysize, 1, band.DataType) 
        #set new raster metadata 
        dst_ds.SetProjection(raster_srs.ExportToWkt())
        dst_ds.SetGeoTransform(geoT)


        blocks = 0


        for y in range(0, ysize, y_block_size):
            #print blocks
            if y + y_block_size < ysize:
                rows = y_block_size
            else:
                rows = ysize - y
            for x in range(0, xsize, x_block_size):
                if x + x_block_size < xsize:
                    cols = x_block_size
                else:
                    cols = xsize - x
                array = band.ReadAsArray(x, y, cols, rows)

                #calculate slope and aspect algorithm
                slope_array, aspect_array =  self.get_slope_and_aspect(array = dem_array)

                #DO ALGORITHM
                #calculate tpi values
                """add calculation to pad blocks by max tpi value and squize export tif by number of pixels equal to max tpi"""
                """do i even need an export raster? i think not! but i should squize the tif by max tpi"""
                tpi_array_dict = {}
                for tpi_radius in self.TPI_Radius_m_lst:
                    tpi_array_dict[f"tpi_{tpi_radius}"] = self.calculate_TPI(dem_array = dem_array, radius_m = tpi_radius ,resolution = resolution)
                    """#use the threads only to run the tpi algorithm,
                    x = t.map(partial(self.calculate_TPI, dem_array = dem_array,resolution = resolution),radius_m = self.TPI_Radius_m_ls)"""
                #export values as sets to dataframe.
                

                #EXPORT EACH BLOCK INDIVIDUALLY TO EXPORT RASTER
                dst_ds.GetRasterBand(1).WriteArray(array, x, y)
                del array
                blocks += 1
        
        return lf_array ,legend_dict
        """" #calculate slope and aspect algorithm
            slope_array, aspect_array =  self.get_slope_and_aspect(array = dem_array)

            #calculate tpi values
            tpi_array_dict = {}
            for tpi_radius in self.TPI_Radius_m_lst:
                tpi_array_dict[f"tpi_{tpi_radius}"] = self.calculate_TPI(dem_array = dem_array, radius_m = tpi_radius ,resolution = resolution)
                use the threads only to run the tpi algorithm,
                x = t.map(partial(self.calculate_TPI, dem_array = dem_array,resolution = resolution),radius_m = self.TPI_Radius_m_ls

            #calculate landform classes
            lf_array ,legend_dict = self.add_prime_labels(tpi_array_dict, slope_array)

        
        return lf_array ,legend_dict""""


UnicodeEncodeError: 'charmap' codec can't encode character '\u03a3' in position 7798: character maps to <undefined>

In [102]:
Test = TPI_Land_Form_Classification(elevation_model = "ALASKA/Alaska.tif", TPI_Radius_m_lst = [10,20])
lf_array ,legend_dict = Test.test_fit()

1:52: E251 unexpected spaces around keyword / parameter equals
1:54: E251 unexpected spaces around keyword / parameter equals
1:80: E501 line too long (102 > 79 characters)
1:92: E251 unexpected spaces around keyword / parameter equals
1:94: E251 unexpected spaces around keyword / parameter equals
1:98: E231 missing whitespace after ','
2:9: E203 whitespace before ','
2:10: E231 missing whitespace after ','


# Test Zone

In [ ]:
"""def discrete_matshow(data):
    # get discrete colormap
    cmap = plt.get_cmap('RdBu', np.max(data) - np.min(data) + 1)
    # set limits .5 outside true range
    mat = plt.matshow(data, cmap=cmap, vmin=np.min(data) - 0.5, 
                      vmax=np.max(data) + 0.5)
    # tell the colorbar to tick at integers
    cax = plt.colorbar(mat, ticks=np.arange(np.min(data), np.max(data) + 1))"""

In [ ]:
cmap = plt.get_cmap('terrain',len(np.unique(lf_array)))
plt.imshow(lf_array, cmap = cmap)
plt.title("lf_array")
plt.colorbar()
plt.show()

In [1]:
tpi_10 = np.genfromtxt("tpi_10.csv" , delimiter= ",")
tpi_30 = np.genfromtxt("tpi_30.csv" , delimiter= ",")
tpi_50 = np.genfromtxt("tpi_50.csv" , delimiter= ",")
tpi_100 = np.genfromtxt("tpi_100.csv" , delimiter= ",")
tpi_300 = np.genfromtxt("tpi_300.csv" , delimiter= ",")
tpi_500 = np.genfromtxt("tpi_500.csv" , delimiter= ",")

slope =  np.genfromtxt("slope.csv" , delimiter= ",")
aspect = np.genfromtxt("aspect.csv" , delimiter= ",")
tpi_array_dict = { "tpi_10":tpi_10, "tpi_30":tpi_30, "tpi_50":tpi_50, "tpi_100":tpi_100, "tpi_300":tpi_300, "tpi_500":tpi_500}

KeyboardInterrupt: 

In [ ]:
#get raster base data
ds = gdal.Open(elevation_model)
#get raster and data
band = dem_data.GetRasterBand(1)

# Get "natural" block size, and total raster XY size. 
block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]

#add band and array size
xsize = band.XSize
ysize = band.YSize

#get the geotransform data from the raster
geoT = dem_data.GetGeoTransform()
#get projection data
projection = dem_data.GetProjection()
#get the rasters resolution
resolution = geoT[1]

#get projection reference
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(ds.GetProjectionRef())
#----------------------------------------------------------------
#set export data
# set driver
driver = gdal.GetDriverByName('GTiff')
#set output driver and instance new raster
dst_ds = driver.Create(output_model, xsize, ysize, 1, band.DataType) 
#set new raster metadata 
dst_ds.SetProjection(raster_srs.ExportToWkt())
dst_ds.SetGeoTransform(geoT)


blocks = 0
    
    
for y in range(0, ysize, y_block_size):
    #print blocks
    if y + y_block_size < ysize:
        rows = y_block_size
    else:
        rows = ysize - y
    for x in range(0, xsize, x_block_size):
        if x + x_block_size < xsize:
            cols = x_block_size
        else:
            cols = xsize - x
        array = band.ReadAsArray(x, y, cols, rows)

        #DO ALGORITHM
        #add calculation that removes empty values and makes sure that each block is the right size.
        
        
        

        #EXPORT EACH BLOCK INDIVIDUALLY TO EXPORT RASTER
        dst_ds.GetRasterBand(1).WriteArray(array, x, y)
        del array
        blocks += 1

#remove writen raster from memory
dst_ds.FlushCache()
band = None
ds = None
dst_ds = None

In [ ]:
import rasterio
from rasterio.features import shapes
mask = None
with rasterio.Env():
    with rasterio.open('a_raster') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)
# first feature
print( geoms[0])


In [270]:
from shapely.geometry import shape
shape(geoms[0]['geometry'])

NameError: name 'geoms' is not defined

In [127]:
from math import gcd
# Function to print all the
# common divisors
def getAllDivisors(arr):
    
    # Variable to find the gcd
    # of N numbers
    g = arr[0]
    N = len(arr)
 
    # Set to store all the
    # common divisors
    #divisors = dict()
    divisors = set()    
    # Finding GCD of the given
    # N numbers
    print(2)
    for i in range(1, N):
        g = gcd(arr[i], g)
       
    # Finding divisors of the
    # HCF of n numbers
    for i in range(1, g + 1):
        if i*i > g:
            break
            divisors.add(i)
        if (g % i == 0):
            divisors.add(i)
            #divisors[i] = 1
            if (g // i != i):
                #divisors[g // i] = 1
                divisors.add(g // i)
    #return set of all devisors
    return divisors

4:1: E302 expected 2 blank lines, found 0
5:1: W293 blank line contains whitespace
10:1: W293 blank line contains whitespace
13:5: E265 block comment should start with '# '
14:21: W291 trailing whitespace
20:1: W293 blank line contains whitespace
29:13: E265 block comment should start with '# '
31:17: E265 block comment should start with '# '
33:5: E265 block comment should start with '# '


In [263]:
# np.split(array , 5)
getAllDivisors([5846, 2916])

{1, 2}

In [265]:
# shorten array by minimum number of pixels relevant for calculation
# ADD TO DOCUMENTATION THAT THERE IS A LOSS OF 2000M ON THE BOUNDRY 
# OF THE DEM ITERATE OVER BLOCKS TO GET ALL TPI VALUES PERTAINING TO SPECIFIC 
# MAJOR LANDFORM AND EXPORT THE GEOMETRYS 
# COULD EVEN DO IT DURING THE CALCULATION 
# THE MINUTE THE tpi IS CALCULATED FOR LARGEST VALUE EXPORT THE GEOMETRYS AND 
# UNIFY THE OVERLAPING ONES WITH THE SAME TYPE)
# USE POLYGON MASK ON THE TPI ARRAY AND GENERATE THE DIFFRENT HEIRARCHYS UNDER 
# THIS INDIVIDUAL LANDFORM ALONG WITH THIER GEOMETRY and a hashmap/set of the coordinate values they contain 
# for quick lookup times.
# add overland stuff as properties of smallest polygons
# add additional overland stuff properties
# save the tiff as a gdal geotiff and lookup the blocksizes
# add instance geotiff

2:68: W291 trailing whitespace
3:78: W291 trailing whitespace
4:42: W291 trailing whitespace
5:42: W291 trailing whitespace
6:78: W291 trailing whitespace
8:79: W291 trailing whitespace
9:80: E501 line too long (108 > 79 characters)
9:109: W291 trailing whitespace
14:1: E265 block comment should start with '# '


In [266]:
# save each diffrent base landform as an individual shapefile

In [268]:
#get raster base data
dem_data = gdal.Open("ALASKA/Alaska.tif")
#get raster and data
dem_band = dem_data.GetRasterBand(1)

# Get "natural" block size, and total raster XY size. 
block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]

#add band and array size
xsize = band.XSize
ysize = band.YSize

#get the geotransform data from the raster
transform = dem_data.GetGeoTransform()
#get projection data
projection = dem_data.GetProjection()
#get the rasters resolution
resolution = transform[1]

AttributeError: 'NoneType' object has no attribute 'GetBlockSize'

1:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
9:54: W291 trailing whitespace
16:1: E265 block comment should start with '# '
18:1: E265 block comment should start with '# '
20:1: E265 block comment should start with '# '


In [247]:
def read_raster_blocks(x_block_size, y_block_size, elevation_model):
    # Function to read the raster as arrays for the chosen block size.
    ds = gdal.Open(elevation_model)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    blocks = 0
    
    
    for y in range(0, ysize, y_block_size):
        #print blocks
        if y + y_block_size < ysize:
            rows = y_block_size
        else:
            rows = ysize - y
        for x in range(0, xsize, x_block_size):
            if x + x_block_size < xsize:
                cols = x_block_size
            else:
                cols = xsize - x
            array = band.ReadAsArray(x, y, cols, rows)
            
            #DO ALGORITHM
            
            
            
            #EXPORT EACH BLOCK INDIVIDUALLY TO EXPORT RASTER
            dst_ds.GetRasterBand(1).WriteArray(array, x, y)
            del array
            blocks += 1

import numpy
from numpy import zeros
from numpy import logical_and
from osgeo import gdal, osr
#import struct


















#get raster base data
ds = gdal.Open(elevation_model)
#get raster and data
band = dem_data.GetRasterBand(1)

# Get "natural" block size, and total raster XY size. 
block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]

#add band and array size
xsize = band.XSize
ysize = band.YSize

#get the geotransform data from the raster
geoT = dem_data.GetGeoTransform()
#get projection data
projection = dem_data.GetProjection()
#get the rasters resolution
resolution = geoT[1]

#get projection reference
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(ds.GetProjectionRef())
#----------------------------------------------------------------
#set export data
# set driver
driver = gdal.GetDriverByName('GTiff')
#set output driver and instance new raster
dst_ds = driver.Create(output_model, xsize, ysize, 1, band.DataType) 
#set new raster metadata by old raster metadata
dst_ds.SetProjection(raster_srs.ExportToWkt())
dst_ds.SetGeoTransform(geoT)


blocks = 0
    
    
for y in range(0, ysize, y_block_size):
    #print blocks
    if y + y_block_size < ysize:
        rows = y_block_size
    else:
        rows = ysize - y
    for x in range(0, xsize, x_block_size):
        if x + x_block_size < xsize:
            cols = x_block_size
        else:
            cols = xsize - x
        array = band.ReadAsArray(x, y, cols, rows)

        #DO ALGORITHM



        #EXPORT EACH BLOCK INDIVIDUALLY TO EXPORT RASTER
        dst_ds.GetRasterBand(1).WriteArray(array, x, y)
        del array
        blocks += 1

#remove writen raster from memory
dst_ds.FlushCache()
band = None
ds = None
dst_ds = None

dst_ds.GetRasterBand(1).WriteArray(out)
#remove writen raster from memory
dst_ds.FlushCache()
ds = None
------------------------------------------
#print x_block_size, y_block_size
driver = gdal.GetDriverByName( "GTiff" )
dst_ds = driver.Create("output.tif", xsize, ysize, 1, band.DataType )
dst_ds.SetGeoTransform(geoT)
dst_ds.SetProjection(raster_srs.ExportToWkt())

#print"working................... 
# Function to read the raster as arrays for the chosen block size.
            
            
read_raster(x_block_size, y_block_size)
band = None
ds = None
dst_ds = None

#print"............... Done................."

5:1: E265 block comment should start with '# '
11:1: E265 block comment should start with '# '
15:5: E225 missing whitespace around operator
17:54: W291 trailing whitespace
24:1: E265 block comment should start with '# '
26:31: E201 whitespace after '('
26:38: E202 whitespace before ')'
27:77: E202 whitespace before ')'
32:1: E265 block comment should start with '# '
32:34: W291 trailing whitespace
34:1: E302 expected 2 blank lines, found 1
42:9: E265 block comment should start with '# '
53:13: E265 block comment should start with '# '
54:13: E265 block comment should start with '# '
55:24: E225 missing whitespace around operator
55:27: E225 missing whitespace around operator
56:17: E116 unexpected indentation (comment)
56:17: E265 block comment should start with '# '
57:13: E265 block comment should start with '# '
58:17: E116 unexpected indentation (comment)
58:17: E265 block comment should start with '# '
62:1: E265 block comment should start with '# '
64:1: E305 expected 2 blank li